In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [25]:
data = pd.read_csv('customer_churn.csv')
data.head()

,CustomerID,Tenure,MonthlyCharges,TotalCharges,Contract,PaymentMethod,Churn
0,1001,5,70.0,350.0,Month-to-month,Electronic check,1
1,1002,10,85.5,850.5,Two year,Mailed check,0
2,1003,3,55.3,165.9,One year,Electronic check,1
3,1004,8,90.0,720.0,Month-to-month,Credit card,0
4,1005,2,65.2,130.4,One year,Electronic check,1


In [31]:
data = pd.get_dummies(data, drop_first=True)

In [32]:
X = data.drop('Churn', axis=1)
y = data['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
class ChurnModel(nn.Module):
    def __init__(self):
        super(ChurnModel, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = nn.functional.dropout(x, 0.5, training=self.training)
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

model = ChurnModel()

In [34]:
# Convert categorical variables to dummy variables
X_train_dummies = X_train.astype(np.float32)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (simplified example)
for epoch in range(10):
    model.train()
    optimizer.zero_grad()
    outputs = model(torch.tensor(X_train_dummies.values, dtype=torch.float32))
    loss = criterion(outputs.squeeze(), torch.tensor(y_train.values, dtype=torch.float32))
    loss.backward()
    optimizer.step()

In [35]:
X_test = X_test.astype(np.float32)

model.eval()
outputs = model(torch.tensor(X_test.values, dtype=torch.float32))
predictions = (outputs.squeeze().detach().numpy() > 0.5).astype(int)
accuracy = np.mean(predictions == y_test.values)
print(f'Test accuracy: {accuracy}')

Test accuracy: 0.0


In [37]:
# Apply dynamic quantization
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

In [38]:
torch.save(model.state_dict(), 'churn_model.pth')